## Assignment 1
### Senan Gaffori, 20949022
### Ayush Bhargava, 20889700

### 1.1 - An optimization problem with Gurobipy

In [7]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("DemoExample")

x = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x")
y = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="y")

m.setObjective(x + y, GRB.MAXIMIZE)

m.addConstr(x+2*y <=4, "c1")
m.addConstr(x*4+3*y <= 12, "c2")

m.optimize()

if m.status == GRB.OPTIMAL:
    if m.status == GRB.OPTIMAL:
        print(f"Objective Value: {m.objVal}")
        print(f"x: {x.X}, y: {y.X}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 24.4.0 24E263)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x9a331afa
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+01]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+30   3.250000e+30   2.000000e+00      0s
       2    3.2000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.200000000e+00
Objective Value: 3.2
x: 2.4, y: 0.8


### 1.2 - An optimization problem with CPLEX

In [ ]:
%pip install docplex

from docplex.mp.model import Model

# Create a model
mdl = Model(name='DemoDocplex')

# Variables
x = mdl.continuous_var(name='x', lb=0)
y = mdl.continuous_var(name='y', lb=0)

# Objective: Maximize x + y
mdl.maximize(x + y)

# Constraints
mdl.add_constraint(x + 2*y <= 4, 'c1')
mdl.add_constraint(x*4 + 3*y <= 12, 'c2')

# Solve
sol = mdl.solve(log_output=True)

# Output
if sol:
    print(f"Objective Value: {mdl.objective_value}")
    mdl.print_solution()
else:
    print("No solution found")

From 1.1 and 1.2 we see the final solution has optimal values of:
x = 2.4
y = 0.8
z = 3.2

## 2 - Lab Assignment

## Problem Formulation

## CFLP - Sets, Parameters and Decsion Variables

### Sets:

$$
I: \text{ Set of facilities } (f_1, f_2, f_3, f_4, f_5, f_6, f_7)
$$
$$
J: \text{ Set of customers } (c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8, c_9, c_{10})
$$

### Parameters:
$$
f_i: \text{ Fixed cost of opening facility } i \in I
$$
$$
u_i: \text{ Capacity of facility } i \in I
$$
$$
d_j: \text{ Demand of customer } j \in J
$$
$$
c_{ij}: \text{ Transportation cost of shipping from facility } i \text{ to customer } j =\$3.5 * Distance
$$\
$$
r: \text{ Revenue earned per unit of demand fulfilled } = \$1000
$$

### Decision Variables:
$$
y_i \in \{0,1\}: \text{ Binary variable indicating if facility } i \text{ is open}
$$
$$
x_{ij} \geq 0: \text{ Amount shipped from facility } i \text{ to customer } j
$$\
$$
z_{ij} \in \{0, 1\}: 1 \text{ if customer } j \text{ is assigned to facility } i, 0 \text{ otherwise}
$$

## Objective Function

$$
\text{Maximize } z =  \sum_{i \in I} \sum_{j \in J} r \cdot x_{ij} - \sum_{i \in I} f_i y_i - \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}
$$

## Constraints

**Each customer is served by at most one facility:**
$$
\sum_{i \in I} z_{ij} \leq 1 \quad \forall j \in J
$$

**Shipping allowed only if customer is assigned to that facility:**
$$
x_{ij} \leq d_j \cdot z_{ij} \quad \forall i \in I, \forall j \in J
$$

**A facility cannot exceed its capacity:**
$$
\sum_{j \in J} x_{ij} \leq u_i \cdot y_i \quad \forall i \in I
$$

**Customer can only be assigned to an open facility:**
$$
z_{ij} \leq y_i \quad \forall i \in I, \forall j \in J
$$

**At most 3 facilities may be opened:**
$$
\sum_{i \in I} y_i \leq 3
$$

**Variable domains:**
$$
y_i \in \{0, 1\}, \quad z_{ij} \in \{0, 1\}, \quad x_{ij} \geq 0
$$


# Code Implementation w/ Gurobi

## Import Libaries, Read CSV file w/ Customer Data, and Create Cost of Transportation Matrix

In [1]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# --- Load CSV ---
df = pd.read_csv("LabData.csv")

# --- Extract facility and customer data ---
facility_data = df[['Facility', 'Fixed Cost ($)', 'Capacity (units)']].dropna().reset_index(drop=True)
facilities = facility_data['Facility'].tolist()
fixed_cost = dict(zip(facility_data['Facility'], facility_data['Fixed Cost ($)']))
capacity = dict(zip(facility_data['Facility'], facility_data['Capacity (units)']))

customer_cols = df.columns[df.columns.get_loc('Facility.1')+1:]
customers = customer_cols.tolist()

customer_data = df[['Customer', 'Demand (units)']].dropna().reset_index(drop=True)
demand = dict(zip(customer_data['Customer'], customer_data['Demand (units)']))

# --- Calculate transport costs from distances ---
# Assumes distances are in the main df, in rows indexed by 'Facility' and columns named by customers.
# The 3.5 is the shipping cost per unit of distance.
shipping_rate = 3.5
transport_costs_matrix = {}

# Prepare a DataFrame view for easier lookup of distances.

facility_rows_for_distances_df = df[df['Facility'].isin(facilities)].copy()
facility_rows_for_distances_df.set_index('Facility', inplace=True)

for i in facilities:  # i is facility name, e.g., 'F1'
    transport_costs_matrix[i] = {}
    for j in customers:  # j is customer name, e.g., 'C1'
        try:
            distance = facility_rows_for_distances_df.loc[i, j]
            if pd.isna(distance) or str(distance).strip() == "":
                print(f"Warning: Missing or invalid distance for Facility {i} to Customer {j}. Found '{distance}'. Using a high placeholder cost.")
                # Using a very high cost to effectively prevent shipping if distance is missing/invalid
                transport_costs_matrix[i][j] = 999999 * shipping_rate
            else:
                transport_costs_matrix[i][j] = float(distance) * shipping_rate
        except KeyError:
            print(f"Critical Error: Data inconsistency. Could not find distance entry for Facility '{i}' / Customer '{j}'.")
            print(f"Ensure Facility '{i}' (from facilities list) has a corresponding row in the CSV's distance matrix section,")
            print(f"and Customer '{j}' (from customer list derived from CSV headers) is a column in that section.")
            transport_costs_matrix[i][j] = 9999999 * shipping_rate # Prohibitive cost
        except ValueError:
            print(f"Error: Non-numeric distance value for Facility {i}, Customer {j}: '{facility_rows_for_distances_df.loc[i, j]}'. Using high placeholder cost.")
            transport_costs_matrix[i][j] = 9999999 * shipping_rate # Prohibitive cost

## Build Model w/ Constraints and Assumptions

In [2]:
# --- Gurobi Model ---
m = Model("FacilityLocation_Lab1")

# Parameters
revenue_per_unit = 1000  # Assumed constant revenue per unit shipped
# transport_cost_per_unit = 3.5  # Fixed transportation cost per unit shipped (REPLACED)

# Decision variables
y = m.addVars(facilities, vtype=GRB.BINARY, name="Open")           # Open facility
x = m.addVars(facilities, customers, vtype=GRB.CONTINUOUS, name="Ship")  # Quantity shipped
z = m.addVars(facilities, customers, vtype=GRB.BINARY, name="Assign")    # Assignment

# --- Objective: Maximize profit ---
# Profit = revenue - fixed cost - transport cost
m.setObjective(
    quicksum(revenue_per_unit * x[i, j] for i in facilities for j in customers)
    - (quicksum(fixed_cost[i] * y[i] for i in facilities)
    + quicksum(transport_costs_matrix[i][j] * x[i, j] for i in facilities for j in customers)),
    GRB.MAXIMIZE
)

# --- Constraints ---

# 1. Each customer must be assigned to exactly one facility
for j in customers:
    m.addConstr(quicksum(z[i, j] for i in facilities) == 1, name=f"AssignOnce_{j}")

# 2. Shipping only allowed if assigned
for i in facilities:
    for j in customers:
        m.addConstr(x[i, j] <= demand[j] * z[i, j], name=f"ShipIfAssigned_{i}_{j}")

# 2b. If assigned, must ship at least 1 unit (if demand allows - see note in code)
# This ensures that if z[i,j] is 1, then x[i,j] must be at least 1.
# This may cause infeasibility if demand[j] < 1 for any customer j.
for i in facilities:
    for j in customers:
        m.addConstr(x[i, j] >= z[i, j], name=f"MinShipmentIfAssigned_{i}_{j}")

# 3. Facility capacity not exceeded
for i in facilities:
    m.addConstr(quicksum(x[i, j] for j in customers) <= capacity[i] * y[i], name=f"Capacity_{i}")

# 4. Facility must be open to serve
for i in facilities:
    for j in customers:
        m.addConstr(z[i, j] <= y[i], name=f"ServeIfOpen_{i}_{j}")

# 5. At most 3 facilities can be open
m.addConstr(quicksum(y[i] for i in facilities) <= 3, name="Max3Facilities")

# --- Solve ---
m.optimize()


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 24.4.0 24E263)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 228 rows, 147 columns and 574 nonzeros
Model fingerprint: 0xc2df76b1
Variable types: 70 continuous, 77 integer (77 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [9e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 96383.500000
Presolve removed 70 rows and 0 columns
Presolve time: 0.01s
Presolved: 158 rows, 147 columns, 504 nonzeros
Variable types: 0 continuous, 147 integer (77 binary)

Root relaxation: objective 1.290605e+05, 100 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I

## Print Solution

In [6]:
if m.status == GRB.OPTIMAL:
    print(f"\nTotal Profit: {m.objVal:.2f}")
    
    open_facilities_details = {}
    # First, identify open facilities and prepare for detailed output
    for i in facilities:
        if y[i].x > 0.5:
            open_facilities_details[i] = {"name": i, "ships_to": [], "total_shipped_from_facility": 0}

    print("\n--- Customer Assignments and Shipments ---")
    all_customers_fully_served = True
    customers_not_fully_served_details = []

    for j in customers: # For each customer
        assigned_facility_name = None
        shipped_quantity_to_j = 0
        customer_j_is_assigned = False

        for i in facilities: # Check all facilities for assignment to customer j
            if z[i, j].x > 0.5: # If customer j is assigned to facility i
                customer_j_is_assigned = True
                assigned_facility_name = i
                shipped_quantity_to_j = x[i, j].x
                
                if assigned_facility_name in open_facilities_details: # This facility i must be open
                    open_facilities_details[assigned_facility_name]["ships_to"].append(
                        {"customer": j, "quantity": shipped_quantity_to_j}
                    )
                    open_facilities_details[assigned_facility_name]["total_shipped_from_facility"] += shipped_quantity_to_j
                else:
                    # This case should ideally not happen if z[i,j]=1 implies y[i]=1 (Constraint 4)
                    print(f"WARNING: Customer {j} assigned to facility {i} (z[{i},{j}].x = {z[i,j].x:.2f}), but facility {i} is marked closed (y[{i}].x = {y[i].x:.2f}). Check ServeIfOpen constraint.")
                break # Customer j found its one assigned facility

        customer_demand_j = demand.get(j, 0) # Get demand, default to 0 if not found
        if customer_j_is_assigned:
            print(f"Customer {j}: Assigned to Facility {assigned_facility_name}, Shipped: {shipped_quantity_to_j:.1f}, Demand: {customer_demand_j:.1f}")
            if abs(shipped_quantity_to_j - customer_demand_j) > 0.001 and shipped_quantity_to_j < customer_demand_j:
                print(f"  WARNING: Customer {j} demand NOT fully met (Shipped: {shipped_quantity_to_j:.1f} / Demand: {customer_demand_j:.1f}).")
                all_customers_fully_served = False
                customers_not_fully_served_details.append(f"Customer {j} (Demand: {customer_demand_j:.1f}, Shipped: {shipped_quantity_to_j:.1f})")
            if shipped_quantity_to_j < 0.001 and customer_demand_j > 0:
                print(f"  INFO: Customer {j} assigned but received no significant shipment. Possible reasons: high transport cost, facility capacity reached.")
                # This customer contributes to the "left out" feeling if they don't get goods
                if all_customers_fully_served: # Avoid double flag if already not fully served
                     all_customers_fully_served = False # Technically assigned, but effectively left out if no shipment
                if f"Customer {j}" not in str(customers_not_fully_served_details):
                    customers_not_fully_served_details.append(f"Customer {j} (Demand: {customer_demand_j:.1f}, Shipped: {shipped_quantity_to_j:.1f} - effectively no shipment)")


    print("\n--- Facility Shipment Summary ---")
    for fac_id in facilities:
        if y[fac_id].x > 0.5: # If facility is open
            details = open_facilities_details[fac_id]
            print(f"\nFacility {details['name']} is OPEN. Capacity: {capacity.get(fac_id, 'N/A')}. Total Shipped from here: {details['total_shipped_from_facility']:.1f}")
            if details["ships_to"]:
                for shipment_info in details["ships_to"]:
                    if shipment_info['quantity'] > 0.001: # Only detail actual shipments
                        print(f"  Ships {shipment_info['quantity']:.1f} units to Customer {shipment_info['customer']}")
            else:
                print(f"  Ships to no customers (or all shipment quantities were zero).")

    if not all_customers_fully_served:
        print("\n--- Summary of Service Issues ---")
        print("One or more customers were not fully served or had issues with their assignment:")
        for detail in customers_not_fully_served_details:
            print(f"- {detail}")
            
elif m.status == GRB.INFEASIBLE:
    print("\nModel is INFEASIBLE.")
    

else:
    print(f"\nOptimization stopped with status code {m.status}. Solution may not be optimal or feasible.")


Total Profit: 129053.50

--- Customer Assignments and Shipments ---
Customer C1: Assigned to Facility F2, Shipped: 18.0, Demand: 18.0
Customer C2: Assigned to Facility F6, Shipped: 22.0, Demand: 22.0
Customer C3: Assigned to Facility F6, Shipped: 3.0, Demand: 25.0
Customer C4: Assigned to Facility F6, Shipped: 30.0, Demand: 30.0
Customer C5: Assigned to Facility F5, Shipped: 27.0, Demand: 27.0
Customer C6: Assigned to Facility F2, Shipped: 2.0, Demand: 20.0
Customer C7: Assigned to Facility F5, Shipped: 15.0, Demand: 15.0
Customer C8: Assigned to Facility F2, Shipped: 19.0, Demand: 19.0
Customer C9: Assigned to Facility F5, Shipped: 23.0, Demand: 23.0
Customer C10: Assigned to Facility F2, Shipped: 21.0, Demand: 21.0

--- Facility Shipment Summary ---

Facility F2 is OPEN. Capacity: 60.0. Total Shipped from here: 60.0
  Ships 18.0 units to Customer C1
  Ships 2.0 units to Customer C6
  Ships 19.0 units to Customer C8
  Ships 21.0 units to Customer C10

Facility F5 is OPEN. Capacity: 6

## Final Solution


**Total Profit:** $129053.50$

**Customer Assignments and Shipments**
* Customer C1: Assigned to Facility F2, Shipped: $18.0$, Demand: $18.0$
* Customer C2: Assigned to Facility F6, Shipped: $22.0$, Demand: $22.0$
* Customer C3: Assigned to Facility F6, Shipped: $3.0$, Demand: $25.0$ **(Demand NOT fully met)**
* Customer C4: Assigned to Facility F6, Shipped: $30.0$, Demand: $30.0$
* Customer C5: Assigned to Facility F5, Shipped: $27.0$, Demand: $27.0$
* Customer C6: Assigned to Facility F2, Shipped: $2.0$, Demand: $20.0$ **(Demand NOT fully met)**
* Customer C7: Assigned to Facility F5, Shipped: $15.0$, Demand: $15.0$
* Customer C8: Assigned to Facility F2, Shipped: $19.0$, Demand: $19.0$
* Customer C9: Assigned to Facility F5, Shipped: $23.0$, Demand: $23.0$
* Customer C10: Assigned to Facility F2, Shipped: $21.0$, Demand: $21.0$

**Summary of Service Issues**
* One or more customers were not fully served or had issues with their assignment:
    * Customer C3 (Demand: $25.0$, Shipped: $3.0$)
    * Customer C6 (Demand: $20.0$, Shipped: $2.0$)

## GenAI Disclosure

GenAI and LLMs were used to format the text in the jupyter notebook to be a user friendly reading form.
